### Imports

In [1]:
from colossus.cosmology import cosmology
from matplotlib import pyplot as plt
import numpy as np

import AGNCatalogToolbox as act
import ACTLiterature as actl 

### Processing

In [ ]:
cosmo = 'planck18'
cosmology = cosmology.setCosmology(cosmo)

z = 0
h = cosmology.H0/100

print("Reading in Catalog")
effective_halo_mass, effective_z, virial_mass, up_id, x_coord, y_coord, z_coord = act.load_halo_catalog(h, z, cosmology, "MD_")

print("halo mass to stellar mass")
stellar_mass = act.halo_mass_to_stellar_mass(effective_halo_mass, effective_z)

print("stellar mass to black hole mass")
black_hole_mass = act.stellar_mass_to_black_hole_mass(stellar_mass, method="Shankar16")

print("duty cycle")
duty_cycle = act.to_duty_cycle("Man16", stellar_mass, black_hole_mass)

print("luminosity")
luminosity = act.black_hole_mass_to_luminosity(black_hole_mass, duty_cycle, stellar_mass, z)

print("nh")
nh = act.luminosity_to_nh(luminosity, z)

print("agn_type")
agn_type = act.nh_to_type(nh)

print("wp")
wp_plotting_data = act.compute_wp(x_coord, y_coord, z_coord, 1000/h, weights=duty_cycle)

Reading in Catalog
Loading Halo Catalogue
Found file: MD_0.093.npy
dtypes found:  [('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('id', '<i8'), ('upid', '<i8'), ('mvir', '<f4'), ('Acc_Scale', '<f4'), ('Macc', '<f4')]
    Sorting list w.r.t. upId
    copying all 114235673 elements with up_id = -1
    sorting remaining list list w.r.t. main id
    copying remaining 16470020 elements
halo mass to stellar mass
stellar mass to black hole mass
duty cycle
luminosity
nh
agn_type


/media/chris/Files/EuclidAGN/AGNCatalogToolbox.py:635: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  type[(nh >= 22) * (nh < 24)] = type2[[(nh >= 22) * (nh < 24)]]


wp


In gridlink_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 46,46,29.  Time taken =   4.343 sec
countpairs_wp_float> gridlink seems inefficient. nmesh = (46, 46, 29); avg_np = 2.13e+03. Boosting bin refine factor - should lead to better performance
xmin = 0.000000 xmax=1477.978149 rpmax = 31.622777
In gridlink_float> Running with [nmesh_x, nmesh_y, nmesh_z]  = 93,93,29.  Time taken =   4.019 sec
Using AVX2 kernel
0%.........10%........

### Plot WP

In [ ]:
plt.figure()

wp_data = actl.WPData(z)


plt.plot(wp_plotting_data.x, wp_plotting_data.y, ':')

plt.errorbar(wp_data.r_Koutoulidis, wp_data.wp_Koutoulidis,\
                yerr = wp_data.wp_Koutoulidis_e, fmt='o', label = "Koutoulidis Data")

bins = np.logspace(-1, 1.5, 50)
plt.plot(bins, wp_data.k_power_law(bins), label = "Koutoulidis Fit")
plt.title(r'wp, z = {}'.format(z), fontname = 'Times New Roman')    
plt.xlabel(r'$r_p$ $Mpc$')
plt.ylabel(r'$w(r_p)$')    

plt.legend()
plt.loglog()